# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Pre-Processing, re-map text to categorical numeric

In [5]:
# Create our features
X = df.drop(['loan_status'], axis=1)

# Create our target
y = df['loan_status'].to_frame()

In [6]:
#get distinct column data types
X.dtypes.value_counts()

float64    76
object      9
dtype: int64

In [7]:
#check for distinct values for each of the text columns
sel_cols = list(X.select_dtypes(include='object'))

for c in sel_cols:
    print(X[c].value_counts(), '\n')

MORTGAGE    36219
RENT        24613
OWN          7346
ANY           639
Name: home_ownership, dtype: int64 

Not Verified       32895
Source Verified    25737
Verified           10185
Name: verification_status, dtype: int64 

Jan-2019    31041
Feb-2019    25579
Mar-2019    12197
Name: issue_d, dtype: int64 

n    68817
Name: pymnt_plan, dtype: int64 

w    60292
f     8525
Name: initial_list_status, dtype: int64 

May-2019    42449
Apr-2019    26368
Name: next_pymnt_d, dtype: int64 

Individual    59206
Joint App      9611
Name: application_type, dtype: int64 

N    68817
Name: hardship_flag, dtype: int64 

N    68817
Name: debt_settlement_flag, dtype: int64 



In [8]:
# Create our features, drop issues date, next_payment dates, and flags of constant value 
X = X.drop(['issue_d', 'next_pymnt_d', 'pymnt_plan', 'hardship_flag', 'debt_settlement_flag'], axis=1)

#one-hot encode home_ownership
dummy = pd.get_dummies(X['home_ownership'])
X = pd.concat([X, dummy], axis=1).drop('home_ownership', axis=1)

#one-hot encode verification_status
dummy = pd.get_dummies(X['verification_status'])
X = pd.concat([X, dummy], axis=1).drop('verification_status', axis=1)

#convert to binary: initial_list_status & application_type
X['initial_list_status'] = pd.factorize(X['initial_list_status'])[0]
X['application_type'] = pd.factorize(X['application_type'])[0]

In [9]:
X.dtypes.value_counts()

float64    76
uint8       7
int64       2
dtype: int64

In [10]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk       68470
high_risk        347
dtype: int64

In [11]:
# Convert y to categorical numeric
y['loan_status'] = pd.factorize(y['loan_status'])[0]
y.value_counts()

loan_status
0              68470
1                347
dtype: int64

## Split the Data into Training and Testing

In [12]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y['loan_status'], random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [13]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [14]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [15]:
# Scale the training and testing data
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [16]:
# Train BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

model = BalancedRandomForestClassifier(random_state=1, sampling_strategy=1.0)
model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1, sampling_strategy=1.0)

In [17]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7191048472246664

In [18]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[14269,  2835],
       [   40,    61]], dtype=int64)

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.83      0.60      0.91      0.71      0.52     17104
          1       0.02      0.60      0.83      0.04      0.71      0.49       101

avg / total       0.99      0.83      0.61      0.90      0.71      0.52     17205



In [20]:
# get the features importances
importances = model.feature_importances_

# Sort the feature importance in descending order
sorted_indices = np.argsort(importances)[::-1]

#print column names in order of descending feature importance
X.columns[sorted_indices]

Index(['total_rec_prncp', 'total_pymnt_inv', 'total_rec_int', 'total_pymnt',
       'last_pymnt_amnt', 'int_rate', 'dti', 'annual_inc', 'il_util',
       'max_bal_bc', 'bc_util', 'total_bal_ex_mort', 'installment',
       'total_bc_limit', 'mths_since_recent_inq', 'bc_open_to_buy',
       'revol_bal', 'mths_since_recent_bc', 'out_prncp_inv', 'tot_hi_cred_lim',
       'total_rev_hi_lim', 'out_prncp', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'tot_cur_bal', 'mths_since_rcnt_il',
       'total_acc', 'total_il_high_credit_limit', 'avg_cur_bal',
       'num_rev_accts', 'total_bal_il', 'all_util', 'inq_last_12m',
       'loan_amnt', 'num_il_tl', 'open_il_24m', 'mo_sin_rcnt_tl',
       'mo_sin_rcnt_rev_tl_op', 'num_bc_tl', 'open_acc', 'num_bc_sats',
       'num_tl_op_past_12m', 'open_rv_24m', 'total_rec_late_fee',
       'num_op_rev_tl', 'num_sats', 'open_act_il', 'percent_bc_gt_75',
       'num_actv_bc_tl', 'inq_fi', 'num_rev_tl_bal_gt_0', 'pct_tl_nvr_dlq',
       'num_actv_rev_tl

### Easy Ensemble Classifier

In [21]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

model = EasyEnsembleClassifier(random_state=1, sampling_strategy=1.0)
model.fit(X_train, y_train)

EasyEnsembleClassifier(random_state=1, sampling_strategy=1.0)

In [22]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7158634654391538

In [23]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[12634,  4470],
       [   31,    70]], dtype=int64)

In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.74      0.69      0.85      0.72      0.51     17104
          1       0.02      0.69      0.74      0.03      0.72      0.51       101

avg / total       0.99      0.74      0.69      0.84      0.72      0.51     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Balanced Random Forest Classifier had a better balanced accuracy score than Easy Ensemble Classifier (0.7191 vs 0.7159).

2. Which model had the best recall score?

    The Balanced Random Forest Classifier had a better recall score than Easy Ensemble Classifier (0.83 vs 0.74).

3. Which model had the best geometric mean score?

    The Easy Ensemble Classifier had a better recall score than Balanced Random Forest Classifier (0.72 vs 0.71).

4. What are the top three features?

    The top 3 features were: 'total_rec_prncp', 'total_pymnt_inv' and 'total_rec_int'